In [2]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne
mne.__version__
from mne.viz import plot_alignment, snapshot_brain_montage
import shutil
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split

# from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, make_dataset_description
# from mne_bids.stats import count_events
import sys


In [3]:
path_utils = '/decoding_toolbox_py/helper_funcs' 
sys.path.append(path_utils)

In [4]:
''' VARIABLES '''

dataset = 'eeg'

amount_of_subjects = 1 # Change the range so the process is faster
if amount_of_subjects > 26: amount_of_subjects = 26
subjs_list = ['s{:02d}'.format(i) for i in range(1, amount_of_subjects+1) if i != 6 ] 
print(subjs_list)
nSubj = len(subjs_list)

numC = 8

angles = [i * 180./numC for i in range(numC)]

x_labels = np.array(angles)

resample = False # speeds up the procees but showing worse results overall
if resample: resample_frequency = 20 # in Hz, original freq is 500Hz

cfg_stim = dict()
cfg_stim['kappa'] = 4
cfg_stim['NumC'] = numC
cfg_stim['Tuning'] = 'vonmises'
# cfg_stim['Tuning'] = 'halfRectCos'
cfg_stim['offset'] = 0

cfg_train = dict()
cfg_train['gamma'] = 0.1
cfg_train['demean'] = True
cfg_train['returnPattern'] = True

cfg_test = dict()
cfg_test['demean'] = 'traindata'

['s01']


In [5]:
from toolbox.methods import read_data
all_epochs, all_rawdata = read_data (task = 'main', resample=False,subjs_list=subjs_list, resample_frequency=10, return_epochs =True)

In [6]:
all_epochs[0].ch_names[25]

'Eye'

In [7]:
print(len(all_rawdata))
all_rawdata[0]['metadata'].columns

1


Index(['index', 'subj', 'nblock', 'ntrial', 'nrep', 'trial_type', 'cond-1',
       'cond', 'rDV', 'DV', 'resp', 'deci-2', 'deci-1', 'deci', 'corr-1',
       'r_map', 'correct', 'confi', 'RT', 'd1', 'conf_lvl', 'correct-1', 'd2',
       'd3', 'd4', 'd5', 'd6', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'confi-1',
       'conf_lvl-1'],
      dtype='object')

In [6]:
# all_rawdata[0]['metadata'][['deci','deci-1']].head(50)

In [8]:
all_rawdata[0]['metadata']['correct']

883     1
896     1
909     1
922     0
935     0
       ..
4240    1
4253    1
4266    1
4279    1
4305    1
Name: correct, Length: 250, dtype: int64

In [102]:
use_timepoint = False
reshape = False
timepoint = 10
X = all_rawdata[0]['epoch_dat']
print(X.shape)
# X = X.reshape(X.shape[0],-1)
y = all_rawdata[0]['metadata']['deci']
y = np.array(y)
print(y.shape)

for i in range(1, nSubj):

    X_temp = all_rawdata[i]['epoch_dat']
    # X_temp = X_temp.reshape((X_temp.shape[0]*X_temp.shape[1], X_temp.shape[2]))
    # shapes[i+1] = shapes[i] + X_temp.shape[0]
    y_temp = all_rawdata[i]['metadata']['deci']
    # y_temp = np.tile(y_temp, numC)
    # X = np.vstack((X, X_temp))
    X = np.concatenate((X, X_temp))
    y = np.concatenate((y, y_temp))

print(X.shape, y.shape)
if use_timepoint: 
    X = X[:,:,timepoint]
else:
    if reshape:
        X = X.reshape(X.shape[0], -1)
print(X.shape, y.shape)
X = np.einsum('ijk->ikj',X)
print(X.shape, y.shape)
X = np.delete(X, 25, axis=2)
#X = X[:,:,25]
print(X.shape, y.shape)
X = X.reshape(X.shape[0], -1)
print(X.shape, y.shape)


(250, 32, 2876)
(250,)
(250, 32, 2876) (250,)
(250, 32, 2876) (250,)
(250, 2876, 32) (250,)
(250, 2876, 31) (250,)
(250, 89156) (250,)


In [103]:
# index_0 = [y==0][0]
# index_1 = [y==1][0]
# print(index_0.shape)
# print(index_1.shape)
# index_1[117:250] = False 
# print(y[index_1].shape)
# X[index_0].shape
# index = index_0 + index_1
# y = y[index]
# X = X[index,:,:]
# print(X.shape, y.shape)
# X = np.delete(X, 25, axis=2)
# print(X.shape, y.shape)


In [92]:
from statsmodels.tsa.api import VAR

params = np.array([VAR(X[i]).fit(2).params.flatten() for i in range(X.shape[0])])


In [77]:
print(params.shape)

(250, 2080)


In [80]:
# 
# results_list = []
# results_list_eye = []
# for i in range(X.shape[0]):
#     results = VAR(X[i]).fit(2).params
#     results_eye = results[:,25]
#     results = np.delete(results, 25, axis=1)
#     results_list_eye.append(results_eye)
#     results_list.append(results.flatten())
# 
# params = np.array(results_list)
# params_eye = np.array(results_list_eye)

In [113]:
print(params.shape, params_eye.shape)

(250, 2015) (250, 65)


In [93]:
index_0 = [y==0][0]
index_1 = [y==1][0]
print(index_0.shape)
print(index_1.shape)
index_1[117:250] = False 
print(y[index_1].shape)
index = index_0 + index_1
y = y[index]
params = params[index,:]
print(params.shape, y.shape)
# Xparams = np.delete(params, 25, axis=2)
# print(X.shape, y.shape)

(250,)
(250,)
(81,)
(162, 1953) (162,)


In [104]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming you have defined `params` and `y` before this code snippet

print(np.unique(y, return_counts=True))

# Apply standard scaling
scaler = StandardScaler()
params_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(params_scaled, y, test_size=0.2, random_state=0)

clf = RandomForestClassifier(
    
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(np.unique(y_test, return_counts=True))
print(np.unique(y_pred, return_counts=True))


(array([0, 1]), array([ 81, 169]))
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.69      0.94      0.80        35

    accuracy                           0.66        50
   macro avg       0.34      0.47      0.40        50
weighted avg       0.48      0.66      0.56        50

0.66
(array([0, 1]), array([15, 35]))
(array([0, 1]), array([ 2, 48]))


In [ ]:
arima end

SyntaxError: invalid syntax (3570740957.py, line 1)

In [ ]:
from sklearn.svm import SVC
# from pyrcn.echo_state_network import ESNClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

def train_3(X, y, verbose=False, display_roc=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    clf = CatBoostClassifier(task_type = 'GPU')
    clf = RandomForestClassifier()
    clf = SVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)    
    if verbose:
        print(classification_report(y_test, y_pred))
        print(np.unique(y_test, return_counts=True))
        print(np.unique(y_pred, return_counts=True))
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    if display_roc:
        from sklearn.metrics import roc_curve
        from sklearn.metrics import RocCurveDisplay
        fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=clf.classes_[1])
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
        return roc_display
    return accuracy, f1, roc

In [ ]:
numT = X.shape[2]
timepoint_accuracy = [None]*numT
timepoint_f1 = [None]*numT
timepoint_roc = [None]*numT
for timepoint in range(numT):
    print(timepoint)
    X_training = X[:,:,timepoint]
    timepoint_accuracy[timepoint], timepoint_f1[timepoint], timepoint_roc[timepoint] = train_3(X_training, y)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


KeyboardInterrupt: 

In [ ]:
print('f1 mean: ', np.mean(timepoint_f1))
print('acc mean: ', np.mean(timepoint_accuracy))
plt.plot(timepoint_f1)

# RESULTS

In [ ]:
X_training = X[:,:,1500]
print(train_3(X_training, y, verbose=True))

In [ ]:
from sklearn.svm import SVC
# from pyrcn.echo_state_network import ESNClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# scaler = StandardScaler()

clf = CatBoostClassifier(task_type = 'GPU')
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)



catboost and RF both on more data with stacked are perfoming the same

next im gonna add labels for the subjects


In [ ]:
display(all_rawdata[0]['metadata'].columns)

In [ ]:
print(multivariate)

# multivariate test?

https://pypi.org/project/pyts/

In [ ]:
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

from pyts.datasets import load_basic_motions

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# X_train, X_test, y_train, y_test = load_basic_motions(return_X_y=True)

print(X_train.shape, y_train.shape)

# scaler = StandardScaler()

clf = CatBoostClassifier(task_type = 'GPU')
clf = RandomForestClassifier()
clf = MultivariateClassifier(BOSSVS())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)

